In [57]:
import pandas as pd
import os

In [58]:
month_num_to_name = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

# Data processing

In [59]:
data_base_path ='Data/Raw_airbnb'

In [60]:
# List all files that match the pattern
file_list = [f'{data_base_path}/listings_{i}.csv' for i in range(1, 13)]

# Read and concatenate all CSV files
df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452951 entries, 0 to 452950
Data columns (total 79 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   id                                            452951 non-null  int64  
 1   listing_url                                   452951 non-null  object 
 2   scrape_id                                     452951 non-null  int64  
 3   last_scraped                                  452951 non-null  object 
 4   source                                        452951 non-null  object 
 5   name                                          452927 non-null  object 
 6   description                                   440060 non-null  object 
 7   neighborhood_overview                         251217 non-null  object 
 8   picture_url                                   452940 non-null  object 
 9   host_id                                       45

In [61]:
# Filter relavent columns
df_filtered = df[['id','last_scraped', 'neighbourhood_cleansed','neighbourhood_group_cleansed','latitude', 'longitude', 'room_type', 'accommodates', 'bathrooms', 'bedrooms','beds', 'price', 'minimum_nights','maximum_nights']].copy()

# Change 'last_scraped' to datatime to extract the month column 
df_filtered['last_scraped'] = pd.to_datetime(df_filtered['last_scraped'])
df_filtered['month'] = df_filtered['last_scraped'].dt.month


df_filtered.info()
df_filtered.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452951 entries, 0 to 452950
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   id                            452951 non-null  int64         
 1   last_scraped                  452951 non-null  datetime64[ns]
 2   neighbourhood_cleansed        452951 non-null  object        
 3   neighbourhood_group_cleansed  452951 non-null  object        
 4   latitude                      452951 non-null  float64       
 5   longitude                     452951 non-null  float64       
 6   room_type                     452951 non-null  object        
 7   accommodates                  452951 non-null  int64         
 8   bathrooms                     277436 non-null  float64       
 9   bedrooms                      382977 non-null  float64       
 10  beds                          275689 non-null  float64       
 11  price        

,id,last_scraped,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,month
0,39572,2025-01-03,Hell's Kitchen,Manhattan,40.76203,-73.98869,Private room,2,NaN,1.0,1.0,$139.00,30,1125,1
1,39593,2025-01-03,Sunnyside,Queens,40.74523,-73.92318,Private room,1,1.0,1.0,1.0,$78.00,31,150,1
2,39704,2025-01-03,Bedford-Stuyvesant,Brooklyn,40.68216,-73.94594,Entire home/apt,4,NaN,1.0,NaN,NaN,30,30,1
3,42300,2025-01-03,Lower East Side,Manhattan,40.71992,-73.99167,Entire home/apt,5,2.0,3.0,4.0,$750.00,30,240,1
4,42729,2025-01-03,Carroll Gardens,Brooklyn,40.67813,-74.00062,Private room,2,1.0,1.0,1.0,$185.00,30,60,1


In [62]:
# Filter our listings with high amount of bedrooms and bedrooms 
df_filtered = df_filtered[df_filtered['bedrooms'] <= 4]
df_filtered = df_filtered[df_filtered['bathrooms'] <= 4]
df_filtered = df_filtered[df_filtered['beds'] <= 4]

In [ ]:
#Filter out all listings that has less than 28 days maximum nights (less than a month)
df_filtered = df_filtered[df_filtered['maximum_nights'] >= 28].copy()

# Convert price strings to numbers 
if not pd.api.types.is_numeric_dtype(df_filtered['price']):
    df_filtered['price'] = df_filtered['price'].str.replace('$', '', regex=False).str.replace(',', '')
    df_filtered['price'] = pd.to_numeric(df_filtered['price'], errors='coerce')
    
# New column monthly rent (price * 30 days) 
df_filtered['monthly_price'] = df_filtered['price'] * 30

df_filtered.info()
df_filtered.head()

<class 'pandas.core.frame.DataFrame'>
Index: 258543 entries, 1 to 452950
Data columns (total 16 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   id                            258543 non-null  int64         
 1   last_scraped                  258543 non-null  datetime64[ns]
 2   neighbourhood_cleansed        258543 non-null  object        
 3   neighbourhood_group_cleansed  258543 non-null  object        
 4   latitude                      258543 non-null  float64       
 5   longitude                     258543 non-null  float64       
 6   room_type                     258543 non-null  object        
 7   accommodates                  258543 non-null  int64         
 8   bathrooms                     258543 non-null  float64       
 9   bedrooms                      258543 non-null  float64       
 10  beds                          258543 non-null  float64       
 11  price             

,id,last_scraped,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,month,monthly_price
1,39593,2025-01-03,Sunnyside,Queens,40.74523,-73.92318,Private room,1,1.0,1.0,1.0,78.0,31,150,1,2340.0
3,42300,2025-01-03,Lower East Side,Manhattan,40.71992,-73.99167,Entire home/apt,5,2.0,3.0,4.0,750.0,30,240,1,22500.0
4,42729,2025-01-03,Carroll Gardens,Brooklyn,40.67813,-74.00062,Private room,2,1.0,1.0,1.0,185.0,30,60,1,5550.0
5,43105,2025-01-03,Midtown,Manhattan,40.74490,-73.98499,Entire home/apt,2,1.0,1.0,1.0,165.0,30,365,1,4950.0
6,44229,2025-01-03,Fort Greene,Brooklyn,40.69287,-73.97108,Private room,2,1.0,1.0,1.0,110.0,30,30,1,3300.0


In [65]:
# Save the merged DataFrame to a new CSV file
df_filtered.to_csv('Data/airbnb_shorterm_rental_prices.csv', index=False)

# Analysis

In [66]:
neighborhood_file_path = 'Data/neighbourhoods.csv'

neighborhood_df = pd.read_csv(neighborhood_file_path)

neighborhood_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   neighbourhood_group  230 non-null    object
 1   neighbourhood        230 non-null    object
dtypes: object(2)
memory usage: 3.7+ KB


In [67]:
df = pd.read_csv('Data/airbnb_shorterm_rental_prices.csv')

## Monthly by neighborhood analysis

In [68]:
from itertools import product

# Your expanded list (may include neighborhoods not in original data)
target_neighborhoods = sorted(neighborhood_df['neighbourhood'].tolist())

# Months you want to cover (e.g., 1-12)
target_months = range(1, 13)

# Calculate averages for existing data
avg_prices = (
    df.groupby(['neighbourhood_cleansed', 'month'], as_index=False)['monthly_price']
    .agg(['mean', 'count'])  # Calculate both mean and count
    .round(2)  # Round averages
    .reset_index()
    .rename(columns={'mean': 'avg_monthly_price', 'count': 'listing_count'})
    .astype({'listing_count': int}) 
)

#  Create a complete grid (all neighborhoods × all months)
full_grid = pd.DataFrame(
    list(product(target_neighborhoods, target_months)),
    columns=['neighbourhood_cleansed', 'month']
)

# Merge with averages 
avg_prices_complete = full_grid.merge(
    avg_prices,
    on=['neighbourhood_cleansed', 'month'],
    how='left'
)
avg_prices_complete = avg_prices_complete.drop('index', axis=1)
# avg_prices_complete = avg_prices_complete.rename(columns={'neighbourhood_cleansed': 'neighbourhood'})


In [69]:
avg_prices_complete.to_csv('Data/Monthly_price_by_neighborhood.csv', index=False)

## Monthly by neighborhood based on how many people can accomodate analysis 